# Basic Import

In [1]:
import numpy as np
import meshio
import os
#import tensorflow as tf

In [2]:
import pyvista as pv

# Example of a simple mesh

In [3]:
#mesh_path = 'shapes/shape_plane.vtu'
#mesh_path = 'shapes/shape_deformed.vtu'
mesh_path = 'shapes/shape_deformed_constant.vtu'

### We can check the mesh with pyvista or paraview. Here we chose pyvista to stay in a notebook.

In [4]:
mesh_pv = pv.read(mesh_path)

In [5]:
mesh_pv.plot(show_scalar_bar=True, show_axes=True, notebook=True, show_edges=True, background='w')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Gradient

Let's compute the gradient of this field on this unstructured mesh. We first need to compute (and store to reuse them later) gradient matrix.

In [6]:
mesh = meshio.read(mesh_path,"vtu")

In [7]:
from meshgradient.matrix import build_CON_matrix, build_PCE_matrix, build_AGS_matrix

In [8]:
%time test = build_CON_matrix(mesh)
print()
print('-'*10)
print()
print("Gradient Matrix as a PyTorch Sparse Tensor: ")
print()
print(test)
print()
print('-'*10)
print()

indices : <class 'torch.Tensor'>
indices : torch.Size([2, 30000])
values  : <class 'torch.Tensor'>
values  : torch.Size([30000])
shape   : (5151, 10000)
CPU times: user 2.15 s, sys: 0 ns, total: 2.15 s
Wall time: 2.26 s

----------

Gradient Matrix as a PyTorch Sparse Tensor: 

tensor(indices=tensor([[   0,    0,    1,  ..., 5149, 5150, 5150],
                       [   0,    1,    0,  ..., 9999, 9998, 9999]]),
       values=tensor([0.5000, 0.5000, 0.3295,  ..., 0.3952, 0.5107, 0.4893]),
       size=(5151, 10000), nnz=30000, layout=torch.sparse_coo)

----------



Note that while we used these matrix to perform machine learning training within Tensorflow, you can replace these tensorflow Sparse tensors with any other sparse matrix of your choice.

Let's build and save our 3 gradient matrix: 

In [9]:
import pickle
def build_and_save_gradient_matrix(mesh,gradient_folder,gradient_filename):
  tf_gradient_matrix = build_AGS_matrix(mesh)
  with open(os.path.join(gradient_folder,'AGS_' + gradient_filename), 'wb') as handle:
    pickle.dump(tf_gradient_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

  tf_gradient_matrix = build_PCE_matrix(mesh)
  with open(os.path.join(gradient_folder,'PCE_' + gradient_filename), 'wb') as handle:
    pickle.dump(tf_gradient_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

  tf_gradient_matrix = build_CON_matrix(mesh)
  with open(os.path.join(gradient_folder,'CON_' + gradient_filename), 'wb') as handle:
    pickle.dump(tf_gradient_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
build_and_save_gradient_matrix(mesh, 'shapes','example_gradient')

indices : <class 'torch.Tensor'>
indices : torch.Size([2, 89994])
values  : <class 'torch.Tensor'>
values  : torch.Size([89994])
shape   : (15453, 5151)
indices : <class 'torch.Tensor'>
indices : torch.Size([2, 90000])
values  : <class 'torch.Tensor'>
values  : torch.Size([90000])
shape   : (30000, 5151)
indices : <class 'torch.Tensor'>
indices : torch.Size([2, 30000])
values  : <class 'torch.Tensor'>
values  : torch.Size([30000])
shape   : (5151, 10000)


Let's use these matrix to compute our gradients. First we load the 3 tensor and then we use our built in functions to compute the gradient.

In [11]:
def load_gradient_matrix(gradient_folder,gradient_filename):
  with open(os.path.join(gradient_folder,'AGS_' + gradient_filename), 'rb') as handle:
    tf_AGS_matrix = pickle.load(handle)

  with open(os.path.join(gradient_folder,'PCE_' + gradient_filename), 'rb') as handle:
    tf_PCE_matrix = pickle.load(handle)

  with open(os.path.join(gradient_folder,'CON_' + gradient_filename), 'rb') as handle:
    tf_CON_matrix = pickle.load(handle)
  return tf_AGS_matrix, tf_PCE_matrix, tf_CON_matrix

In [12]:
gradient_matrix = load_gradient_matrix('shapes','example_gradient')

We will use 4 boundaries for the 4 sides of our cavity, and compute the gradient of the field "Erreur": 

In [13]:
#b1,b2,b3,b4 = mesh.point_data['AppartientEntree1'],mesh.point_data['AppartientEntree2'],mesh.point_data['AppartientEntree3'],mesh.point_data['AppartientEntree4']

In [14]:
mesh.point_data

{'Erreur': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)}

In [15]:
erreur_field = mesh.point_data['Erreur']

Let's import one of our functions: 

In [16]:
from meshgradient.gradient_fn import compute_gradient_per_points

In [17]:
#gradient_erreur = compute_gradient_per_points(gradient_matrix,erreur_field,b1,b2,b3,b4)

In [18]:
gradient_erreur = compute_gradient_per_points(gradient_matrix,erreur_field)

In [23]:
gradient_erreur.abs().sum()

tensor(0.)

Let's add this field to our mesh, and display it (or compare it in paraview with the built in gradient from paraview).

In [20]:
mesh.point_data['Erreur_grad'] = gradient_erreur.numpy()
meshio.write('shapes/shape_with_gradient.vtu',mesh)

In [21]:
mesh_pv = pv.read('shapes/shape_with_gradient.vtu')

In [22]:
mesh_pv.plot(show_scalar_bar=True, show_axes=True, notebook=True, show_edges=True, background='w')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)